<a href="https://colab.research.google.com/github/CrosbyRd/SPEA-algorithms/blob/main/VRPTW_SPEA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install paretoset

In [ ]:
import random
import os
import io
import math
from paretoset import paretoset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
def calcular_distancia(customer1, customer2):
    return math.sqrt((customer1['coord']['x'] - customer2['coord']['x'])**2 + (customer1['coord']['y'] - customer2['coord']['y'])**2)

In [ ]:
def preprocesamiento(file):
    text_file = f'{file}.txt'
    data = {}
    with io.open(text_file, 'rt', encoding='utf-8', newline='') as file_object:
        for line_count, line in enumerate(file_object, start=1):
            if line_count in [1, 3, 5]:
                pass
            elif line_count == 2:
                values = line.strip().split()
                data['customers'] = int(values[0])
            elif line_count == 4:
                values = line.strip().split()
                data['vehicle_capacity'] = float(values[0])
            elif line_count == 6:
                values = line.strip().split()
                data['origin'] = {
                    'coord': {
                        'x': float(values[1]),
                        'y': float(values[2]),
                    },
                    'demand': float(values[3]),
                    'ready_time': float(values[4]),
                    'due_time': float(values[5]),
                    'service_time': float(values[6]),
                }
            else:
                values = line.strip().split()
                data[f'cust_{values[0]}'] = {
                    'coord': {
                        'x': float(values[1]),
                        'y': float(values[2]),
                    },
                    'demand': float(values[3]),
                    'ready_time': float(values[4]),
                    'due_time': float(values[5]),
                    'service_time': float(values[6]),
                }
    customers = ['origin'] + [f'cust_{x}' for x in range(1, 101)]
    data['distance_matrix'] = [[calcular_distancia(data[customer1], data[customer2]) for customer1 in customers] for customer2 in customers]
    return data

In [ ]:
def individuoRuta(individuo, instancia):
    ruta = []
    capacidad = instancia['vehicle_capacity']
    tiempo_limite = instancia['origin']['due_time']
    for cust_id in individuo:
        instancia[f'cust_{cust_id}']['visited'] = 0
    while todosVisitados(individuo, instancia) == False:
        sub_ruta = []
        carga_vehiculo = 0
        tiempo_transcurrido = 0
        ultimo_cust_id = 0
        for cust_id in individuo:
            if instancia[f'cust_{cust_id}']['visited'] != 1:
                distancia = instancia['distance_matrix'][ultimo_cust_id][cust_id]
                demanda = instancia[f'cust_{cust_id}']['demand']
                carga_actual = carga_vehiculo + demanda
                tiempo_llegada = tiempo_transcurrido + distancia
                tiempo_espera = max(instancia[f'cust_{cust_id}']['ready_time'] - tiempo_llegada, 0)
                tiempo_servicio = instancia[f'cust_{cust_id}']['service_time']
                if carga_actual > capacidad:
                    continue
                if tiempo_llegada + tiempo_servicio > instancia[f'cust_{cust_id}']['due_time']:
                    continue
                sub_ruta.append(cust_id)
                carga_vehiculo += demanda
                tiempo_transcurrido = tiempo_llegada + tiempo_espera + tiempo_servicio
                instancia[f'cust_{cust_id}']['visited'] = 1
                ultimo_cust_id = cust_id
        if sub_ruta != []:
                ruta.append(sub_ruta)
    return ruta

def todosVisitados(individuo, instancia):
    count = 0
    for cust_id in individuo:
        if instancia[f'cust_{cust_id}']['visited'] == 1:
            count += 1
    return count == len(individuo)